In [1]:
def add_edges_to_graph(g, edge_definitions):
    for edge_def in edge_definitions:
        src_type, dst_type, edge_type = edge_def
        src_ids = g.nodes[src_type].data['movie_id'].numpy()  # 取得源節點ID
        dst_ids = g.nodes[dst_type].data['movie_id'].numpy()  # 取得目標節點ID

        # 建立邊的索引對
        for movie_id in src_ids:
            if movie_id in dst_ids:
                src_index = g.nodes[src_type].data['movie_id'].tolist().index(movie_id)
                dst_index = g.nodes[dst_type].data['movie_id'].tolist().index(movie_id)
                g.add_edges(src_index, dst_index, etype=('movie', edge_type, dst_type))

    return g


In [8]:
import os
import pickle
import dgl

def initialize_movie_ids(g):
    # 假设 'movie' 节点已经有 'movie_id'，将其复制给其他相关节点类型
    movie_ids = g.nodes['movie'].data['movie_id']
    for node_type in ['movieimage', 'movieaudio', 'movietext']:
        if node_type in g.ntypes:
            g.nodes[node_type].data['movie_id'] = movie_ids.clone()  # 克隆以避免直接引用

def add_edges_to_graph(g, edge_definitions):
    for edge_def in edge_definitions:
        src_type, dst_type, edge_type = edge_def
        if 'movie_id' in g.nodes[src_type].data and 'movie_id' in g.nodes[dst_type].data:
            src_ids = g.nodes[src_type].data['movie_id'].numpy()
            dst_ids = g.nodes[dst_type].data['movie_id'].numpy()
            src_indices = {mid: idx for idx, mid in enumerate(src_ids)}
            dst_indices = {mid: idx for idx, mid in enumerate(dst_ids)}

            for mid in src_ids:
                if mid in dst_indices:
                    src_index = src_indices[mid]
                    dst_index = dst_indices[mid]
                    g.add_edges(src_index, dst_index, etype=('movie', edge_type, dst_type))
        else:
            print(f"Warning: 'movie_id' not found in '{src_type}' or '{dst_type}'")

def process_graphs(graph_paths, output_directory):
    for graph_path in graph_paths:
        with open(graph_path, 'rb') as f:
            g = pickle.load(f)

        # 初始化 movie_id
        initialize_movie_ids(g)
        
        # 为每个图添加影像、音频和文字的边
        edge_types = [
            ('movie', 'movieimage', 'has_image'),
            ('movie', 'movieaudio', 'has_audio'),
            ('movie', 'movietext', 'has_text')
        ]
        add_edges_to_graph(g, edge_types)

        # 确保输出目录存在
        if not os.path.exists(output_directory):
            os.makedirs(output_directory)
        
        # 构建新的文件路径，保存更新后的图
        base_filename = os.path.basename(graph_path)
        updated_graph_path = os.path.join(output_directory, base_filename.replace('.pkl', '_with_edges.pkl'))
        
        with open(updated_graph_path, 'wb') as f:
            pickle.dump(g, f)
        print(f"Graph saved to {updated_graph_path}")


# 圖檔案的路徑列表
graph_paths = [
    # 'D:\CODE\multi-model knowledge graph multi-graph recommendation system\code\mainmodel\hetero_graph03_with_audio_features.pkl',
    # 'D:\CODE\multi-model knowledge graph multi-graph recommendation system\code\mainmodel\hetero_graph03_with_features_with_audio_features.pkl',
    # 'D:\CODE\multi-model knowledge graph multi-graph recommendation system\code\mainmodel\hetero_graph03_with_features.pkl',
    # 'D:\CODE\multi-model knowledge graph multi-graph recommendation system\code\mainmodel\hetero_graph03_with_text_features_with_audio_features.pkl',
    # 'D:\CODE\multi-model knowledge graph multi-graph recommendation system\code\mainmodel\hetero_graph03_with_text_features.pkl',
    # 'D:\CODE\multi-model knowledge graph multi-graph recommendation system\code\mainmodel\hetero_graph03.pkl',
    # 'D:\CODE\multi-model knowledge graph multi-graph recommendation system\code\mainmodel\hetero_graph05_with_audio_features.pkl',
    # 'D:\CODE\multi-model knowledge graph multi-graph recommendation system\code\mainmodel\hetero_graph05_with_features_with_audio_features.pkl',
    # 'D:\CODE\multi-model knowledge graph multi-graph recommendation system\code\mainmodel\hetero_graph05_with_features.pkl',
    # 'D:\CODE\multi-model knowledge graph multi-graph recommendation system\code\mainmodel\hetero_graph05_with_text_features_with_audio_features.pkl',
    # 'D:\CODE\multi-model knowledge graph multi-graph recommendation system\code\mainmodel\hetero_graph05_with_text_features.pkl',
    # 'D:\CODE\multi-model knowledge graph multi-graph recommendation system\code\mainmodel\hetero_graph05.pkl',
    'D:\CODE\multi-model knowledge graph multi-graph recommendation system\code\mainmodel\hetero_graph08_with_features_with_audio_features.pkl',
    'D:\CODE\multi-model knowledge graph multi-graph recommendation system\code\mainmodel\hetero_graph08_with_features.pkl',
    'D:\CODE\multi-model knowledge graph multi-graph recommendation system\code\mainmodel\hetero_graph08_with_text_features.pkl',
    'D:\CODE\multi-model knowledge graph multi-graph recommendation system\code\mainmodel\hetero_graph08.pkl',

]

output_directory = r'D:\CODE\multi-model knowledge graph multi-graph recommendation system\code\mainmodel\graph'

# 调用函数，处理并保存图
process_graphs(graph_paths, output_directory)

Graph saved to D:\CODE\multi-model knowledge graph multi-graph recommendation system\code\mainmodel\graph\hetero_graph08_with_features_with_audio_features_with_edges.pkl
Graph saved to D:\CODE\multi-model knowledge graph multi-graph recommendation system\code\mainmodel\graph\hetero_graph08_with_features_with_edges.pkl
Graph saved to D:\CODE\multi-model knowledge graph multi-graph recommendation system\code\mainmodel\graph\hetero_graph08_with_text_features_with_edges.pkl
Graph saved to D:\CODE\multi-model knowledge graph multi-graph recommendation system\code\mainmodel\graph\hetero_graph08_with_edges.pkl
